# Census 2020 response rates

## NewsCounts

In [1]:
import pandas as pd
import numpy as np
import requests
import sys

In [36]:
states = pd.read_csv("Data/state_codes.txt",sep=" 	", engine='python')
counties = pd.read_csv("Data/county_codes.txt",sep=" 	", engine='python')

In [3]:
key = "124e1b1d5ceca3b46b3f6bd97d95ea8d4d780e28"

In [22]:
url = f"https://api.census.gov/data/2020/dec/responserate?get=RESP_DATE,GEO_ID,DRRALL,CRRALL&for=state:*&key={key}"
response = requests.get(url)
if response:
    print('State: Success!')
else:
    print('State: An error has occurred.')
    sys.exit("State: "+response.text)

State: Success!


Check that there is a new data available.

In [23]:
state_df = pd.DataFrame.from_records(response.json()[1:])
state_df.columns = response.json()[0]

old_df = pd.read_csv("Data/sates_response_rates.csv")
if old_df.shape[0] > 0 :
    last_old = old_df.iloc[old_df.shape[0]-1]
    last = state_df.iloc[state_df.shape[0]-1]
    if last_old.RESP_DATE == last.RESP_DATE:
        print("No new data")
        sys.exit("No new data")
print("New data available!")

New data available!


Create data frame from json.

In [38]:
state_df['state'] = state_df.state.astype(int)

state_df['state_name'] = state_df.state.map(states.set_index('code')['state'])

state_df['state_short'] = state_df.state.map(states.set_index('code')['state_code'])

state_df.sort_values(by="state",inplace=True)
state_df = state_df.reset_index(drop=True)
state_df

,RESP_DATE,GEO_ID,DRRALL,CRRALL,state,state_name,state_short
0,2020-03-20,0400000US01,2.9,15.1,1,Alabama,AL
1,2020-03-20,0400000US02,1.3,7.6,2,Alaska,AK
2,2020-03-20,0400000US04,2.9,14.9,4,Arizona,AZ
3,2020-03-20,0400000US05,2.5,14.9,5,Arkansas,AR
4,2020-03-20,0400000US06,2.3,14.7,6,California,CA
5,2020-03-20,0400000US08,3.2,14.5,8,Colorado,CO
6,2020-03-20,0400000US09,3.3,13.2,9,Connecticut,CT
7,2020-03-20,0400000US10,3,14.6,10,Delaware,DE
8,2020-03-20,0400000US11,2.3,14.8,11,District of Columbia,DC
9,2020-03-20,0400000US12,2.4,14.3,12,Florida,FL


In [28]:
state_df.to_csv("Data/sates_response_rates.csv",mode = 'a',header = False, index = False)

I do the same for all counties:

In [6]:
data=[]
url = f"https://api.census.gov/data/2020/dec/responserate?get=RESP_DATE,GEO_ID,DRRALL,CRRALL&for=county:*&key={key}"
response = requests.get(url)
if response:
    print('County: Success!')
    data = response.json()
else:
    print('An error has occurred.')
    sys.exit("County: "+response.text)

County: Success!


In [8]:
county_df = pd.DataFrame.from_records(data[1:])
county_df.columns = data[0]
county_df['state_county'] = county_df.state + county_df.county
county_df['state_county'] = county_df.state_county.astype(int)
county_df['state'] = county_df.state.astype(int)

county_df['state_name'] = county_df.state.map(states.set_index('code')['state'])
county_df['state_short'] = county_df.state.map(states.set_index('code')['state_code'])

county_df['county_name'] = county_df.state_county.map(counties.set_index('code')['county'])

county_df.sort_values(by=["state_county"],inplace=True)
county_df = county_df.reset_index(drop=True)
county_df

,RESP_DATE,GEO_ID,DRRALL,CRRALL,state,county,state_county,state_name,state_short,county_name
0,2020-03-20,0500000US01001,3.9,18.6,1,001,1001,Alabama,AL,Autauga
1,2020-03-20,0500000US01003,2.7,11.9,1,003,1003,Alabama,AL,Baldwin
2,2020-03-20,0500000US01005,2.1,13.5,1,005,1005,Alabama,AL,Barbour
3,2020-03-20,0500000US01007,1.3,16.2,1,007,1007,Alabama,AL,Bibb
4,2020-03-20,0500000US01009,2.3,16.6,1,009,1009,Alabama,AL,Blount
...,...,...,...,...,...,...,...,...,...,...
3211,2020-03-20,0500000US72145,0,0,72,145,72145,Puerto Rico,PR,Vega Baja
3212,2020-03-20,0500000US72147,0,0,72,147,72147,Puerto Rico,PR,Vieques
3213,2020-03-20,0500000US72149,0,0,72,149,72149,Puerto Rico,PR,Villalba
3214,2020-03-20,0500000US72151,0,0,72,151,72151,Puerto Rico,PR,Yabucoa


In [9]:
county_df.to_csv("Data/counties_response_rates.csv",mode = 'a', header= False, index = False)

I do the same for all tracts:

In [14]:
data=[]
c=0
for i in range(len(state_df)):
    s = state_df.iloc[i,:]
    if s.state <10:
        num = f"0{s.state}"
    else:
        num = s.state
    url = f"https://api.census.gov/data/2020/dec/responserate?get=RESP_DATE,GEO_ID,DRRALL,CRRALL&for=tract:*&in=state:{num}&key={key}"
    response = requests.get(url)
    if  not response:
        print('Tract: An error has occurred.')
        print(response.text)
        sys.exit("Tract: "+response.text)
    else:
        start = 0 if c==0 else 1
        data = data + response.json()[start:]
        c=c+1
        
print("Tract: Done!")

Tract: Done!


In [15]:
tract_df = pd.DataFrame.from_records(data[1:])
tract_df.columns = data[0]
tract_df['state_county'] = tract_df.state + tract_df.county
tract_df['state_county'] = tract_df.state_county.astype(int)
tract_df['state'] = tract_df.state.astype(int)

tract_df['state_name'] = tract_df.state.map(states.set_index('code')['state'])
tract_df['state_short'] = tract_df.state.map(states.set_index('code')['state_code'])

tract_df['county_name'] = tract_df.state_county.map(counties.set_index('code')['county'])

tract_df.sort_values(by=["state_county","tract"],inplace=True)
tract_df = tract_df.reset_index(drop=True)
tract_df

,RESP_DATE,GEO_ID,DRRALL,CRRALL,state,county,tract,state_county,state_name,state_short,county_name
0,2020-03-20,1400000US01001020100,5,8,1,001,020100,1001,Alabama,AL,Autauga
1,2020-03-20,1400000US01001020200,1.7,21.7,1,001,020200,1001,Alabama,AL,Autauga
2,2020-03-20,1400000US01001020300,4.7,20.8,1,001,020300,1001,Alabama,AL,Autauga
3,2020-03-20,1400000US01001020400,1.8,31.6,1,001,020400,1001,Alabama,AL,Autauga
4,2020-03-20,1400000US01001020501,2.4,28.7,1,001,020501,1001,Alabama,AL,Autauga
...,...,...,...,...,...,...,...,...,...,...,...
84088,2020-03-20,1400000US72153750501,0,0,72,153,750501,72153,Puerto Rico,PR,Yauco
84089,2020-03-20,1400000US72153750502,0,0,72,153,750502,72153,Puerto Rico,PR,Yauco
84090,2020-03-20,1400000US72153750503,0,0,72,153,750503,72153,Puerto Rico,PR,Yauco
84091,2020-03-20,1400000US72153750601,0,0,72,153,750601,72153,Puerto Rico,PR,Yauco


In [16]:
tract_df.to_csv("Data/tracts_response_rates.csv",mode='a', header = False,  index = False)

In [37]:
states

,state,state_code,code
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6
5,Colorado,CO,8
6,Connecticut,CT,9
7,Delaware,DE,10
8,District of Columbia,DC,11
9,Florida,FL,12
